In [260]:
import argparse
import csv
import itertools
import logging
import time

import requests
import yaml
from bs4 import BeautifulSoup

logging.root.setLevel(logging.INFO)

In [262]:
with open('env.yml', 'r') as stream:
    ENV = yaml.safe_load(stream)
print(ENV)

{'ARTICLE_COUNT_SPAN': 'lx-pagination__page-number qa-pagination-total-page-number', 'HEADLINE_SPAN_CLASS_A': 'bbc-13dm3d0 e1yj3cbb0', 'HEADLINE_SPAN_CLASS_B': None, 'STORY_DIV_CLASS': 'bbc-19j92fr e57qer20', 'CATEGORY_URLS': {'MOST_POPULAR': 'https://www.bbc.com/pidgin/popular/read', 'NIGERIA': 'https://www.bbc.com/pidgin/topics/c2dwqd1zr92t', 'AFRICA': 'https://www.bbc.com/pidgin/topics/c404v061z85t', 'WORLD': 'https://www.bbc.com/pidgin/topics/c0823e52dd0t', 'SPORT': 'https://www.bbc.com/pidgin/topics/cjgn7gv77vrt', 'ENTERTAINMENT': 'https://www.bbc.com/pidgin/topics/cqywjyzk2vyt'}}


In [263]:
ENV['CATEGORY_URLS']

{'MOST_POPULAR': 'https://www.bbc.com/pidgin/popular/read',
 'NIGERIA': 'https://www.bbc.com/pidgin/topics/c2dwqd1zr92t',
 'AFRICA': 'https://www.bbc.com/pidgin/topics/c404v061z85t',
 'WORLD': 'https://www.bbc.com/pidgin/topics/c0823e52dd0t',
 'SPORT': 'https://www.bbc.com/pidgin/topics/cjgn7gv77vrt',
 'ENTERTAINMENT': 'https://www.bbc.com/pidgin/topics/cqywjyzk2vyt'}

In [ ]:
aa = list(ENV['CATEGORY_URLS'].keys())
str(aa).upper().split(",")


In [268]:
ALL_CATEGORIES = ENV["CATEGORY_URLS"] 

In [264]:
ALL_CATEGORIES.items()

dict_items([('NIGERIA', 'https://www.bbc.com/pidgin/topics/c2dwqd1zr92t'), ('AFRICA', 'https://www.bbc.com/pidgin/topics/c404v061z85t'), ('WORLD', 'https://www.bbc.com/pidgin/topics/c0823e52dd0t'), ('VIDEO', 'https://www.bbc.com/pidgin/media/video'), ('SPORT', 'https://www.bbc.com/pidgin/topics/cjgn7gv77vrt'), ('ENTERTAINMENT', 'https://www.bbc.com/pidgin/topics/cqywjyzk2vyt')])

In [207]:
for category, urls in ALL_CATEGORIES.items():
  # urls = str(urls.split(" "))
  print(list(urls.split(" ")))
  print(urls)
  # for url in urls:
  #   print(url)

['https://www.bbc.com/pidgin/popular/read']
https://www.bbc.com/pidgin/popular/read
['https://www.bbc.com/pidgin/topics/c2dwqd1zr92t']
https://www.bbc.com/pidgin/topics/c2dwqd1zr92t
['https://www.bbc.com/pidgin/topics/c404v061z85t']
https://www.bbc.com/pidgin/topics/c404v061z85t
['https://www.bbc.com/pidgin/topics/c0823e52dd0t']
https://www.bbc.com/pidgin/topics/c0823e52dd0t
['https://www.bbc.com/pidgin/media/video']
https://www.bbc.com/pidgin/media/video
['https://www.bbc.com/pidgin/topics/cjgn7gv77vrt']
https://www.bbc.com/pidgin/topics/cjgn7gv77vrt
['https://www.bbc.com/pidgin/topics/cqywjyzk2vyt']
https://www.bbc.com/pidgin/topics/cqywjyzk2vyt


In [265]:
def get_parser() -> argparse.ArgumentParser:
    """
    parse command line arguments

    returns:
        parser - ArgumentParser object
    """
    parser = argparse.ArgumentParser(description="BBC Pidgin Scraper")
    parser.add_argument(
        "--output_file_name",
        type=str,
        default="bbc_pidgin_corpus.csv",
        help="Name of output file",
    )
    parser.add_argument(
        "--categories",
        type=list,
        default=list(ENV['CATEGORY_URLS'].keys()),
        help="This is a list of keys of the categories"
    )
    parser.add_argument(
        "--time_delay",
        # type=bool,
        # action="store_false",
        default=True,
        help="Time delay is set to true by default"
    )
    parser.add_argument(
        "--no_of_articles",
        type=int,
        default=10,
        help="This is the number of articles"
    )
    # TODO: Add your other command line arguments!
    #   Be sure to include their types & any default values.
    return parser

In [266]:
parser = get_parser()
params, unknown = parser.parse_known_args()
print(params)

Namespace(output_file_name='bbc_pidgin_corpus.csv', categories=['MOST_POPULAR', 'NIGERIA', 'AFRICA', 'WORLD', 'SPORT', 'ENTERTAINMENT'], time_delay=True, no_of_articles=10)


In [72]:
val = "Demehin, Ibukun"
val.upper().split(",")

['DEMEHIN', ' IBUKUN']

In [ ]:
categories = params.categories.upper().split(",")
print(categories)

In [ ]:
if params.categories != "all":
    categories = params.categories.upper().split(",")
    categories = {category: ALL_CATEGORIES[category] for category in categories}
else:
    categories = ALL_CATEGORIES

In [269]:
if params.categories:
    categories = params.categories
    categories = {category: ALL_CATEGORIES[category] for category in categories}
else:
    categories = ALL_CATEGORIES
print(categories)

{'MOST_POPULAR': 'https://www.bbc.com/pidgin/popular/read', 'NIGERIA': 'https://www.bbc.com/pidgin/topics/c2dwqd1zr92t', 'AFRICA': 'https://www.bbc.com/pidgin/topics/c404v061z85t', 'WORLD': 'https://www.bbc.com/pidgin/topics/c0823e52dd0t', 'SPORT': 'https://www.bbc.com/pidgin/topics/cjgn7gv77vrt', 'ENTERTAINMENT': 'https://www.bbc.com/pidgin/topics/cqywjyzk2vyt'}


In [224]:
for category, urls in categories.items():
  print(urls)
  # for url in urls:
  #   print(url)

https://www.bbc.com/pidgin/topics/cjgn7gv77vrt


In [270]:
def get_page_soup(url: str): # ENV['CATEGORY_URLS']['NIGERIA']
    """
    Makes a request to a url and creates a beautiful soup object from the response html

    input:
        :param url: input page url
    returns:
        - page_soup: beautiful soup object from the response html
    """

    response = requests.get(url)
    page_soup = BeautifulSoup(response.text, "html.parser")
    # TODO: Complete the function to parse and return the page
    return page_soup

In [ ]:
page_soup = get_page_soup(categories['SPORT'])
print(page_soup)

In [271]:
def get_valid_urls(category_page: BeautifulSoup):
    """
    Gets all valid urls from a category page

    input:
        :param: url: category_page
    returns:
        - valid_urls: list of all valid article urls on a given category page
    """
    base_url = "https://www.bbc.com"

    all_urls = category_page.findAll("a")
    valid_article_urls = []
    for url in all_urls:
        
        href = url.get("href")
        if (href.startswith("/pidgin/tori") or \
            href.startswith("/pidgin/world") or \
            href.startswith("/pidgin/sport")):
            complete_href = f"{base_url}{href}"
            valid_article_urls.append(complete_href)
        
        # from a look at BBC pidgin's urls, they always begin with the following strings.
        # so we obtain valid article urls using these strings
    return list(set(valid_article_urls))

In [228]:
get_valid_urls(page_soup)


['https://www.bbc.com/pidgin/tori-58692696',
 'https://www.bbc.com/pidgin/sport-58681606',
 'https://www.bbc.com/pidgin/sport-58692695',
 'https://www.bbc.com/pidgin/sport-58744891',
 'https://www.bbc.com/pidgin/sport-58661394',
 'https://www.bbc.com/pidgin/sport-58722442',
 'https://www.bbc.com/pidgin/sport-58736253',
 'https://www.bbc.com/pidgin/sport-58697469',
 'https://www.bbc.com/pidgin/sport-58674414',
 'https://www.bbc.com/pidgin/sport-58750520']

In [253]:
def get_urls(category_url: str, category: str, time_delay: bool) -> list: # CATEGORY_URLS, NIGERIA, 3000
    """
    # TODO: Complete the docstring for this function
    Get all the urls possible on each page
    input:
        :category_url: the list of urlobjects 
        :category: a category that we need to sort the list by 
        :time_delay: A boolean function whether to apply a time delay
    returns:
        - category_urls: list of all urls from the CATEGORY_URLS in the env.yml
    """
    page_soup = get_page_soup(category_url)
    category_urls = get_valid_urls(page_soup)

    # get total number of pages for given category
    article_count_span = page_soup.find_all(
        "span", attrs={"class": "lx-pagination__page-number qa-pagination-total-page-number"}
    )
    # if there are multiple pages, get valid urls from each page
    # else just get the articles on the first page
    if article_count_span:
        total_article_count = int(article_count_span[0].text)
        logging.info(f"{total_article_count} pages found for {category}")
        logging.info(f"{len(category_urls)} urls in page 1 gotten for {category}")

        for count in range(1, total_article_count):
            # TODO: Use your `get_page_soup` and `get_valid_urls` functions
            #   to obtain valid urls from all pages
            next_page = f"{category_url}/page/{count + 1}"
            next_page_soup = get_page_soup(next_page)
            next_category_urls = get_valid_urls(next_page_soup)
            logging.info(f"{len(next_category_urls)} urls in page {count + 1} gotten for {category}")
            category_urls += next_category_urls
            if time_delay:
                time.sleep(10)
    else:
        logging.info(f"Only one page found for {category}. {len(category_urls)} urls gotten")

    return category_urls

In [272]:
def get_urls(category_url: str, category: str, time_delay: bool) -> list: # CATEGORY_URLS, NIGERIA, 3000
    """
    # TODO: Complete the docstring for this function
    Get all the urls possible on each page
    input:
        :category_url: the list of urlobjects 
        :category: a category that we need to sort the list by 
        :time_delay: A boolean function whether to apply a time delay
    returns:
        - category_urls: list of all urls from the CATEGORY_URLS in the env.yml
    """
    page_soup = get_page_soup(category_url)
    category_urls = get_valid_urls(page_soup)

    # print(category_urls)
    # get total number of pages for given category
    article_count_span = page_soup.find_all(
        "span", attrs={"class": "lx-pagination__page-number qa-pagination-total-page-number"}
    )

    
    # if there are multiple pages, get valid urls from each page
    # else just get the articles on the first page
    if article_count_span:
        total_article_count = int(article_count_span[0].text)
        logging.info(f"{total_article_count} pages found for {category}")
        logging.info(f"{len(category_urls)} urls in page 1 gotten for {category}")
    
    

        for count in range(1, 3):
            # TODO: Use your `get_page_soup` and `get_valid_urls` functions
            #   to obtain valid urls from all pages
            next_page = f"{category_url}/page/{count + 1}"
            next_page_soup = get_page_soup(next_page)
            next_category_urls = get_valid_urls(next_page_soup)
            logging.info(f"{len(next_category_urls)} urls in page {count + 1} gotten for {category}")
            category_urls += next_category_urls
            if time_delay:
                time.sleep(10)
    else:
        logging.info(f"Only one page found for {category}. {len(category_urls)} urls gotten")

    return category_urls

In [231]:
get_urls(categories['SPORT'], "Sport", False)

INFO:root:45 pages found for Sport
INFO:root:10 urls in page 1 gotten for Sport
INFO:root:9 urls in page 2 gotten for Sport
INFO:root:10 urls in page 3 gotten for Sport


['https://www.bbc.com/pidgin/tori-58692696',
 'https://www.bbc.com/pidgin/sport-58681606',
 'https://www.bbc.com/pidgin/sport-58692695',
 'https://www.bbc.com/pidgin/sport-58744891',
 'https://www.bbc.com/pidgin/sport-58661394',
 'https://www.bbc.com/pidgin/sport-58722442',
 'https://www.bbc.com/pidgin/sport-58736253',
 'https://www.bbc.com/pidgin/sport-58697469',
 'https://www.bbc.com/pidgin/sport-58674414',
 'https://www.bbc.com/pidgin/sport-58750520',
 'https://www.bbc.com/pidgin/sport-58600151',
 'https://www.bbc.com/pidgin/sport-58466316',
 'https://www.bbc.com/pidgin/sport-58426989',
 'https://www.bbc.com/pidgin/sport-58406381',
 'https://www.bbc.com/pidgin/tori-58479296',
 'https://www.bbc.com/pidgin/sport-58612482',
 'https://www.bbc.com/pidgin/world-58531069',
 'https://www.bbc.com/pidgin/tori-58519837',
 'https://www.bbc.com/pidgin/world-58552521',
 'https://www.bbc.com/pidgin/tori-58318077',
 'https://www.bbc.com/pidgin/sport-58322692',
 'https://www.bbc.com/pidgin/sport-584

In [232]:
article_urls = get_urls(categories['SPORT'], "Sport", False)

INFO:root:45 pages found for Sport
INFO:root:10 urls in page 1 gotten for Sport
INFO:root:9 urls in page 2 gotten for Sport
INFO:root:10 urls in page 3 gotten for Sport


In [273]:
def get_article_data(article_url: str) -> tuple:
    """
    Obtains paragraphs texts and headlines input url article

    input:
        :param article_url: category_page
    returns:
        - headline: headline of url article
        - story_text: text of url article
        - article_url: input article url
    """
    page_soup = get_page_soup(article_url)
    # TODO: Locate the headline element
    headline = page_soup.find(
        "h1", attrs={"class": ENV["HEADLINE_SPAN_CLASS_A"]}
    )
    
    if headline:
        headline = headline.text
        # TODO: Get the headline text from the element
    # return headline

    story_text = " "
    story_div = page_soup.findAll("div", attrs={"class": ENV["STORY_DIV_CLASS"]})    # TODO: Locate all story divs
    if story_div:
        all_paragraphs = [div.findAll("p", recursive=False) for div in story_div]
        all_paragraphs = list(itertools.chain(*all_paragraphs))
        story_text = story_text.join(str(paragraph) for paragraph in all_paragraphs)
        story_text = BeautifulSoup(story_text, "html.parser").get_text()
    story_text = story_text if not story_text == " " else None

    return headline, story_text, article_url

In [234]:
get_article_data(article_urls[2])

('Chelsea vs Man City: Gabriel Jesus goal end Chelsea unbeaten start to Premier League season',
 'Gabriel Jesus second half goal help Manchester City defeat Chelsea 1-0 for dia own backyard. With two strikers on di pitch, Chelsea struggle to create chances as City end dia unbeaten start to di 2021/2022 Premier League season. Jesus deflected shot for di 53rd minute break di Chelsea defence wey don look solid dis season. Jesus almost score second goal wen Thiago Silva clear im effort off di line while Aymeric Laporte for score but im somehow slide e shot wide from inside di six-yard box. Mendy also produce two crucial saves from di excellent Jack Grealish. Before di game, Pep Guardiola bin don lose di last three matches wey im don play against Thomas Tuchel Chelsea including last season Champions League final. Now, Guardiola and im boys don get di better of Chelsea as dem dominate di Blues for Stamford Bridge. More dey come.',
 'https://www.bbc.com/pidgin/sport-58692695')

In [274]:
def scrape(output_file_name: str, no_of_articles: int, category_urls: dict, time_delay: bool) -> None:
    """
    (params.output_file_name, 100, ALL_CATEGORIES, False)

    Main function for scraping and writing articles to file

    input:
        :param output_file_name: file name where output is saved
        :param no_of_articles: number of user specified articles to scrape
        :param category_urls: all articles in a category
    """
    logging.info("Writing articles to file...")

    with open(output_file_name, "w", encoding='utf-8') as csv_file:
        fieldnames = ['Headline', 'Paragraph', "URL"]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        writer.writeheader()
        # TODO: Initialize a csv.DictWriter object
        #   Write the headers your CSV file will take
        story_num = 0

        for category, urls in category_urls.items():
            # print(category)
            logging.info(f"Writing articles for {category} category...")
            for Url in urls:
                # TODO: Get the headline, paragraphs and url using your `get_article_data` func
                headline, paragraphs, url = get_article_data(Url)
                # print(headline)
                if paragraphs:
                    # TODO: Write all the data to CSV
                    writer.writerow({'Headline': headline, 'Paragraph': paragraphs, 'URL': url})
                    story_num += 1
                    logging.info(f"Successfully wrote story number {story_num}")

                if story_num == no_of_articles:
                    logging.info(
                        f"Requested total number of articles {no_of_articles} reached"
                    )
                    logging.info(
                        f"Scraping done. A total of {no_of_articles} articles were scraped!"
                    )
                    return
                if time_delay:
                    time.sleep(10)
    logging.info(
        f"Scraping done. A total of {story_num} articles were scraped!"
    )

In [275]:
category_urls = {}
for category, url in categories.items():
    logging.info(f"Getting all stories for {category}...")
    category_story_links = get_urls(url, category, params.time_delay)
    logging.info(f"{len(category_story_links)} stories found for {category} category")
    category_urls[category] = category_story_links

INFO:root:Getting all stories for MOST_POPULAR...
INFO:root:Only one page found for MOST_POPULAR. 9 urls gotten
INFO:root:9 stories found for MOST_POPULAR category
INFO:root:Getting all stories for NIGERIA...
INFO:root:100 pages found for NIGERIA
INFO:root:9 urls in page 1 gotten for NIGERIA
INFO:root:10 urls in page 2 gotten for NIGERIA
INFO:root:8 urls in page 3 gotten for NIGERIA
INFO:root:27 stories found for NIGERIA category
INFO:root:Getting all stories for AFRICA...
INFO:root:91 pages found for AFRICA
INFO:root:8 urls in page 1 gotten for AFRICA
INFO:root:10 urls in page 2 gotten for AFRICA
INFO:root:10 urls in page 3 gotten for AFRICA
INFO:root:28 stories found for AFRICA category
INFO:root:Getting all stories for WORLD...
INFO:root:100 pages found for WORLD
INFO:root:8 urls in page 1 gotten for WORLD
INFO:root:10 urls in page 2 gotten for WORLD
INFO:root:9 urls in page 3 gotten for WORLD
INFO:root:27 stories found for WORLD category
INFO:root:Getting all stories for SPORT...
I

In [276]:
scrape(params.output_file_name,100,category_urls,False)

INFO:root:Writing articles to file...
INFO:root:Writing articles for MOST_POPULAR category...
INFO:root:Successfully wrote story number 1
INFO:root:Successfully wrote story number 2
INFO:root:Successfully wrote story number 3
INFO:root:Successfully wrote story number 4
INFO:root:Successfully wrote story number 5
INFO:root:Successfully wrote story number 6
INFO:root:Successfully wrote story number 7
INFO:root:Successfully wrote story number 8
INFO:root:Successfully wrote story number 9
INFO:root:Writing articles for NIGERIA category...
INFO:root:Successfully wrote story number 10
INFO:root:Successfully wrote story number 11
INFO:root:Successfully wrote story number 12
INFO:root:Successfully wrote story number 13
INFO:root:Successfully wrote story number 14
INFO:root:Successfully wrote story number 15
INFO:root:Successfully wrote story number 16
INFO:root:Successfully wrote story number 17
INFO:root:Successfully wrote story number 18
INFO:root:Successfully wrote story number 19
INFO:root